In [ ]:
from threading import Thread,Event
import time
def countdown(n,started_evt):
    print('countdown starting')
    started_evt.set()
    while n > 0:
        print('T-minus',n)
        n -= 1
        time.sleep(5)
started_evt = Event()
print('Launching countdown')
t = Thread(target=countdown,args=(10,started_evt))
t.start()
started_evt.wait()
print('countdown is running')

In [ ]:
import threading
import time
class PeriodicTime:
    def __init__(self,interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()
    def start(self):
        t = threading.Thread(target=self.run,daemon=True)
        t.start()
    def run(self):
        while True:
            time.sleep(self._interval)
            with self._cv:
                self._flag ^= 1
                self._cv.notify_all()
    def wait_for_tick(self):
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()
ptimer = PeriodicTime(5)
ptimer.start()
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minux',nticks)
        nticks -= 1
def countup(last):
    n = 0
    while n <last:
        ptimer.wait_for_tick()
        print('counting',n)
        n += 1
threading.Thread(target=countdown,args=(10,)).start()
threading.Thread(target=countup,args=(5,)).start()

In [ ]:
from queue import Queue
from threading import Thread,Event

In [ ]:
class ActorExit(Exception):
    pass
class Actor:
    def __init__(self):
        self._mailbox = Queue()
    def send(self,msg):
        self._mailbox.put(msg)
    def recv(self):
        msg = self._mailbox.get()
        if msg is ActorExit:
            raise ActorExit()
        return msg
    def close(self):
        self.send(ActorExit)
    def start(self):
        self._terminated = Event()
        t = Thread(target=self._bootstrap)
        t.daemon = True
        t.start()
    def _bootstrap(self):
        try:
            self.run()
        except ActorExit:
            pass
        finally:
            self._terminated.set()
    def join(self):
        self._terminated.wait()
    def run(self):
        while True:
            msg = self.recv()
class PrintActor(Actor):
    def run(self):
        while True:
            msg = self.recv()
            print('Got:',msg)

In [ ]:
p = PrintActor()

In [ ]:
p.start()

In [ ]:
p.send('hello')

In [ ]:
p.send('world')

In [ ]:
p.close()

In [ ]:
p.join()

In [24]:
import requests
url = 'http://113.107.161.47:8081/api/counters'
header = {'Content-Type':'application/x-www-form-urlencoded'}
parms = {
    'limit':50000,
    "endpoints":'["113.107.167.139"]'
}
resp = requests.post(url,data=parms,headers=header)
ret = resp.text

In [25]:
dic = json.loads(ret)

In [26]:
len(dic['data'])

589

In [28]:
from urllib import request,parse
url = 'http://httpbin.org/post'
parms = {
    'name1':'value1',
    'name2':'value2'
}
header = {
    'User-agent': 'none/ofyourbusiness',
    'Content-Type':'application/x-www-form-urlencoded',
    'Spam':'Eggs'
}
querystring = parse.urlencode(parms)
req = request.Request(url,querystring.encode('ascii'),headers=header)
u = request.urlopen(req)
u.read()

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Connect-Time": "0", \n    "Connection": "close", \n    "Content-Length": "25", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "Spam": "Eggs", \n    "Total-Route-Time": "0", \n    "User-Agent": "none/ofyourbusiness", \n    "Via": "1.1 vegur", \n    "X-Request-Id": "a38d66c1-2edb-4ef5-9c71-9903b8c40657"\n  }, \n  "json": null, \n  "origin": "39.109.124.40", \n  "url": "http://httpbin.org/post"\n}\n'

In [1]:
%ls

aaa.ipynb          DoubleLinkList.py  ls.spec         stackfun.py
AgentUpgrade.py    du.py              nain.py         t2.py
api.py             find.py            new/            test.py
app.py             funselect.py       Njz.py          timetest.py
argparseme.py      gwc.py             nn/             topheap.py
cat.py             hanoi.py           npoland.py      url.py
cccc.ipynb         juzhen.py          print.py        watch.py
cda.py             leixing.py         __pycache__/    wrap.py
ContextManager.py  LinkList.py        Qpoland.py      yh.py
cp.py              log/               README.md       zuoye/
dist/              ls.py              SortLogFile.py  作业 记录.ipynb


In [2]:
from hashlib import md5

In [3]:
help(md5)

Help on built-in function openssl_md5 in module _hashlib:

openssl_md5(...)
    Returns a md5 hash object; optionally initialized with a string



In [10]:
with open('./cp.py') as f:
    src = f.read()

In [12]:
d = md5(src.encode())

In [16]:
d.hexdigest() == '004f68cca17234e5be9f4a77e8859376'

True

In [17]:
import zipfile

In [20]:
c = zipfile.ZipFile(file='/root/vim.zip')

In [23]:
help(c.extractall)

Help on method extractall in module zipfile:

extractall(path=None, members=None, pwd=None) method of zipfile.ZipFile instance
    Extract all members from the archive to the current working
    directory. `path' specifies a different directory to extract to.
    `members' is optional and must be a subset of the list returned
    by namelist().



In [24]:
c.extractall('/root/testzip')

In [43]:
c.close()

In [37]:
import re

In [ ]:
re.compile

In [25]:
import os

In [26]:
os.system('ls')

0

In [2]:
from urllib import request

In [9]:
a = request.urlopen('http://fe.open-falcon.7road.net:9526/a.txt')

In [10]:
file = a.read()

In [12]:
file.split()[0].decode()

'5ab557c937e38f15291c04b7e99544ad'

In [13]:
import urllib3

ImportError: No module named 'urllib3'

In [34]:
d = a.read()

In [36]:
md5(d).hexdigest() == '69b46bebe9ecf991da18ef2a17dfe816'

True

In [38]:
import time

In [42]:
int(time.time())

1490082544

In [49]:
f = open('./cfg.json')

In [47]:
import json

In [50]:
da = json.loads(f.read())

In [57]:
da['http']

{'backdoor': False, 'enabled': True, 'listen': ':1988'}

In [58]:
da

{'collector': {'ifacePrefix': ['本地连接']},
 'debug': True,
 'heartbeat': {'addr': '113.107.161.84:6030',
  'enabled': True,
  'interval': 60,
  'timeout': 1000},
 'hostname': '',
 'http': {'backdoor': False, 'enabled': True, 'listen': ':1988'},
 'ignore': {'cpu.busy': True},
 'iis': {'enabled': False, 'websites': ['Default Web Site']},
 'ip': '',
 'logfile': 'windows.log',
 'mssql': {'addr': '127.0.0.1',
  'enabled': False,
  'encrypt': 'disable',
  'instance': ['test'],
  'password': '123456',
  'port': 1433,
  'username': 'sa'},
 'transfer': {'addrs': ['113.107.167.9:8433'],
  'enabled': True,
  'interval': 60,
  'timeout': 1000}}